In [ ]:
!nvidia-smi

Fri May 14 11:17:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !git clone https://github.com/airsplay/lxmert.git
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/My Drive/lxmert")

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 133kB 7.0MB/s 
     |████████████████████████████████| 7.5MB 11.6MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
ERROR: botocore 1.20.70 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
#image features: data/vizwiz_imgfeat
#train/validate files: data/vqa

In [ ]:
#use small data to test, make sure no error
!bash run/vqa_finetune.bash 0 vizwiz_classify_tiny --tiny

In [ ]:
#finetune, save best model to vizwiz_all
!bash run/vqa_finetune.bash 0 vizwiz_classify

In [ ]:
#classifier finetune with balanced train dataset
!bash run/vqa_finetune.bash 0 vizwiz_classify_balanced

In [ ]:
!bash run/vqa_test.bash 0 vizwiz_classify_balanced_result --test valid --load snap/vqa/vizwiz_classify_balanced/BEST
#0.729210099606208 LXMERT unbalanced classify
#0.7310632383599722 LXMERT balanced classify

In [ ]:
!bash run/vqa_test.bash 0 vizwiz_all_results --test valid --load snap/vqa/vizwiz_all/BEST
#0.729210099606208 LXMERT classify

train
Load 20523 data from split(s) train.
Start to load Faster-RCNN detected objects from data/vizwiz_imgfeat/36_vizwiz_train.pkl
Loaded 512 images in file data/vizwiz_imgfeat/36_vizwiz_train.pkl in 82 seconds.
Use 444 data in torch dataset
100% 231508/231508 [00:00<00:00, 971166.43B/s]
100% 407873900/407873900 [00:10<00:00, 39245840.17B/s]
LXRT encoder with 9 l_layers, 5 x_layers, and 5 r_layers.
BertAdam Total Iters: 52
Load model from snap/vqa/vizwiz_all/BEST
valid
Load 4319 data from split(s) valid.
Start to load Faster-RCNN detected objects from data/vizwiz_imgfeat/36_vizwiz_val.pkl
Loaded 7748 images in file data/vizwiz_imgfeat/36_vizwiz_val.pkl in 19 seconds.
Use 4317 data in torch dataset
logits results saved
logits results saved
logits results saved
logits results saved
logits results saved
0.4982626824183446


In [ ]:
#validate 
!bash run/vqa_test.bash 0 vizwiz_with_pred_questype --test valid --load snap/vqa/vizwiz_all/BEST
#0.4786889043317073 - without question type
#0.4982626824183446 - with true question type

#unbalanced training dataset
#0.4103544127866542 - with pred question type
#0.420060227009494 - with pred question type if unswerable and other

#0.41864720870974875 - simply adding logits
#0.41568218670372625 - vizwizlogit*1.5 + classify_logits
#0.418160759786886 - vizwizlogit + classify_logits if not number else vizwizlogit
#0.33 - classify_logits if unanswerable else vizwizlogit ??

#simply adding softmax
#0.4116284456798673


train
Load 20523 data from split(s) train.
Start to load Faster-RCNN detected objects from data/vizwiz_imgfeat/36_vizwiz_train.pkl
Loaded 512 images in file data/vizwiz_imgfeat/36_vizwiz_train.pkl in 98 seconds.
Use 444 data in torch dataset
LXRT encoder with 9 l_layers, 5 x_layers, and 5 r_layers.
BertAdam Total Iters: 52
Load model from snap/vqa/vizwiz_all/BEST
valid
Load 4319 data from split(s) valid.
Start to load Faster-RCNN detected objects from data/vizwiz_imgfeat/36_vizwiz_val.pkl
Loaded 7748 images in file data/vizwiz_imgfeat/36_vizwiz_val.pkl in 31 seconds.
Use 4317 data in torch dataset
[]
0.420060227009494


In [ ]:
######################## CODE TO GENERATE FILES BEBLOW  ####################

In [ ]:
# custom Dataset class
class LogitsDataset(Dataset):
    def __init__(self, X, Y = None):
        self.X = X
        if Y is not None:
            self.Y = Y
        else:
            self.Y = None
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.Y is not None:
            return self.X[idx], self.Y[idx]
        else:
            return self.X[idx]

In [ ]:
#elementwise multiplication of two logits
import pickle,json
import numpy as np
PREDICT_LOGIT_PATH = "snap/vqa/vizwiz_classify_result/logit_result.pt"
MAIN_LOGITS_PATH = 'snap/vqa/vizwiz_all_results/logit_result.pt'

with open(PREDICT_LOGIT_PATH, 'rb') as f:
  pred_logits = pickle.load(f)
with open(MAIN_LOGITS_PATH, 'rb') as f:
  main_logits = pickle.load(f)
with open('data/vqa/valid.json') as f1:
  val = json.load(f1)

CLASSIFIER_LABEL2ANS_PATH = "data/vqa/classifier_label2ans.json"
classifier_label2ans = json.load(open(CLASSIFIER_LABEL2ANS_PATH))
QUESTION_TYPE_LABELS_PATH = "data/vqa/trainval_questype2labels.json"
questype2labels = json.load(open(QUESTION_TYPE_LABELS_PATH))
ans2label = json.load(open("data/vqa/trainval_ans2label.json"))
label2ans = json.load(open("data/vqa/trainval_label2ans.json"))

qid2label = {}
for qa in val:
  qid2label[qa["question_id"]] = qa["label"]


fusion_logits = np.zeros((len(pred_logits),48730))
targets = np.zeros((len(pred_logits),len(ans2label)))
ind = 0
for qid,main_l in main_logits.items():
  pred_l = pred_logits[qid]
  classify_map_logit = np.zeros(main_l.size)
  for label in range(0,len(classifier_label2ans)):
    ques_type = classifier_label2ans[label]
    questype_labels = questype2labels[ques_type]
    classify_map_logit[questype_labels] = pred_l[label]
  
  fusion_l = np.multiply(classify_map_logit, main_l)
  fusion_logits[ind] = fusion_l

  label = qid2label[qid]
  target = np.zeros(len(ans2label))
  for ans, score in label.items():
    if ans in ans2label:
      target[ans2label[ans]] = score
  targets[ind] = target
  
  ind += 1
print(fusion_logits.shape)
print(targets.shape)

(4317, 48730)
(4317, 48730)


In [ ]:
train_dataset = LogitsDataset(fusion_logits[:3800], targets[:3800])
val_dataset = LogitsDataset(fusion_logits[3800:], targets[3800:])
train_loader = DataLoader(train_dataset, shuffle = True, batch_size = 8)
val_loader = DataLoader(val_dataset, shuffle = False, batch_size = 8)

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
cuda = torch.cuda.is_available()

# 3CNN uses three
# filter types of sizes 3, 4, and 5, with 100 filters for
# each size.

class MyModel(nn.Module):
    def __init__(self, input_channel, out_channel, kernel_sizes, output_dim):
        super().__init__()
        self.conv1 = nn.Conv1d(input_channel, out_channel, kernel_size = kernel_sizes[0])
        self.conv2 = nn.Conv1d(input_channel, out_channel, kernel_size = kernel_sizes[1])
        self.conv3 = nn.Conv1d(input_channel, out_channel, kernel_size = kernel_sizes[2])
        
        self.cnns = []
        self.cnns.append(self.conv1)
        self.cnns.append(self.conv2)
        self.cnns.append(self.conv3)
        
        self.linear = nn.Linear(len(self.cnns) * out_channel, output_dim)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, embedded):     
        embedded = embedded.permute(0, 2, 1)

        z = [F.relu(conv(embedded)) for conv in self.cnns]
        output = [F.max_pool1d(i, kernel_size=i.size(-1)).squeeze(-1) for i in z]

        output = torch.cat(output, -1)
        output = self.dropout(output)
        output = self.linear(output)
        return output 

input_channel = 100
out_channel = 100
kernel_sizes = [3,4,5]
output_dim = 48730
model = MyModel(input_channel, out_channel, kernel_sizes, output_dim)
device = torch.device("cuda" if cuda else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 5e-5)
model = model.to(device)
model.cuda()

bce_loss = nn.BCEWithLogitsLoss()

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    
    start_time = time.time()
    for batch_idx, (X, Y) in enumerate(train_loader):   
        optimizer.zero_grad()   
        X = X.to(device)
        Y = Y.to(device) 

        outputs = model(X)
        loss = criterion(outputs, Y)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()

    end_time = time.time()
    
    running_loss /= len(train_loader)
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time)
    return running_loss 

def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()


        running_loss = 0.0

        for batch_idx, (X, Y) in enumerate(test_loader):   
            X = X.to(device)
            Y = Y.to(device) 

            outputs = model(X)
            # print(outputs)
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += Y.size(0)
            correct_predictions += (predicted == Y).sum().item()
            
            loss = criterion(outputs, Y)
            running_loss += loss.item()


        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)
        print(all_positive_cnt)
        print(false_negative_cnt)
        precision = true_positive_cnt/all_positive_cnt if all_positive_cnt != 0 else 0
        recall = true_positive_cnt/(true_positive_cnt +false_negative_cnt) if (true_positive_cnt +false_negative_cnt) != 0 else 0
        
        # print('Testing Loss: ', running_loss)
        # print('Testing Accuracy: ', acc, '%')
        print('Testing Precision: ', precision)
        print('Testing Recall: ', recall)
        f1 = 2*(precision*recall/(precision+recall)) if (precision+recall) != 0 else 0
        print('Testing F1: ', f1)
        return running_loss,precision,recall, f1

In [ ]:
n_epochs=1
for i in range(n_epochs):
    print("Epoch: " + str(i+1))
    train_epoch(model, train_loader, criterion, optimizer)
    test_loss, precision,recall, f1 = test_model(model, val_loader, criterion)

In [ ]:
import json
with open('data/vqa/train.json') as f1:
  train = json.load(f1)

for qa in train:
  if qa["question_type"] == "number":
    print(qa["label"])

In [ ]:
#check the classification acc for each question type
import json
with open('snap/vqa/vizwiz_classify_balanced_result/valid_predict.json') as f1:
  pred_qa = json.load(f1)
with open('data/vqa/valid.json')as f1:
  valid = json.load(f1)

print(len(pred_qa))
print(len(valid))
qid2predans = {}
for qa in pred_qa:
  qid2predans[qa["question_id"]] = qa["answer"]

number_total = 0
number_corr = 0
binary_total=0
binary_corr=0
unanswerable_total=0
unanswerable_corr=0
color_total=0
color_corr=0
other_total = 0
other_corr = 0

for qa in valid:
  if qa["question_id"] in qid2predans:
    pred_ans = qid2predans[qa["question_id"]]
    ans = qa["question_type"]
    if pred_ans == "number":
      number_total+=1
      if ans == pred_ans:
        number_corr += 1
    elif pred_ans == "binary":
      binary_total += 1
      if ans == pred_ans:
        binary_corr += 1
    elif pred_ans == "unanswerable":
      unanswerable_total += 1
      if pred_ans == pred_ans:
        unanswerable_corr += 1
    elif pred_ans == "color":
      color_total += 1
      if ans == pred_ans:
        color_corr += 1
    else:
      other_total += 1
      if ans == pred_ans:
        other_corr += 1
print("binary acc: {}".format(binary_corr/binary_total))
print("number acc: {}".format(number_corr/number_total))
print("color acc: {}".format(color_corr/color_total))
print("unanswerable acc: {}".format(unanswerable_corr/unanswerable_total))
print("other acc: {}".format(other_corr/other_total))

4317
4319
binary acc: 0.6888888888888889
number acc: 0.4230769230769231
color acc: 0.7024221453287197
unanswerable acc: 1.0
other acc: 0.7402777777777778


In [ ]:
#classifier: check question type distribution to generate balanced data
import json
with open('data/vqa/train.json') as f1:
  train = json.load(f1)

unanswerable_cnt = 0
color_cnt = 0
number_cnt = 0
binary_cnt = 0
other_cnt = 0
for qa in train:
  question_type = qa["question_type"]
  if question_type == "unanswerable":
    unanswerable_cnt += 1
  elif question_type == "number":
    number_cnt += 1
  elif question_type == "binary":
    binary_cnt += 1
  elif question_type == "other":
    other_cnt += 1
  else:
    color_cnt += 1
print("unanswerable_cnt : {}".format(unanswerable_cnt))
print("color_cnt : {}".format(color_cnt))
print("number_cnt : {}".format(number_cnt))
print("binary_cnt : {}".format(binary_cnt))
print("other_cnt : {}".format(other_cnt))



unanswerable_cnt : 7069
color_cnt : 1731
number_cnt : 406
binary_cnt : 1078
other_cnt : 10239


In [ ]:
#create balanced train
#binary * 10
#color * 6
#number * 25
balanced_train = []
for qa in train:
  question_type = qa["question_type"]
  ori_quesid = qa["question_id"]
  balanced_train.append(qa)
 
  if question_type == "number":  
    for i in range(24):      
      qa["question_id"] = "repeat"+str(i)+"-" + ori_quesid
      balanced_train.append(qa)
  
  elif question_type == "binary":
    for i in range(9):      
      qa["question_id"] = "repeat"+str(i)+"-" + ori_quesid
      balanced_train.append(qa)

  elif question_type == "color":
    for i in range(5):      
      qa["question_id"] = "repeat"+str(i)+"-" + ori_quesid
      balanced_train.append(qa)

with open('data/vqa/train_balanced.json', 'w') as outfile:
    json.dump(balanced_train, outfile)

In [ ]:
#generate classifier's file
import json
classifier_ans2label = {}
classifier_ans2label["binary"] = 0
classifier_ans2label["number"] = 1
classifier_ans2label["unanswerable"] = 2
classifier_ans2label["other"] = 3
classifier_ans2label["color"] = 4

classifier_label2ans = ["binary","number","unanswerable","other","color"]

with open('data/vqa/classifier_ans2label.json', 'w') as outfile:
    json.dump(classifier_ans2label, outfile)

with open('data/vqa/classifier_label2ans.json', 'w') as outfile:
    json.dump(classifier_label2ans, outfile)

In [ ]:
#reformat predicted class type
import json
with open('snap/vqa/vizwiz_classify_result/valid_predict.json') as f1:
  ques_type_pred = json.load(f1)

qid2questype = {}
for qa in ques_type_pred:
  qid2questype[qa["question_id"]] = qa["answer"]

with open('snap/vqa/vizwiz_classify_result/pred_qid2questype.json', 'w') as outfile:
    json.dump(qid2questype, outfile)

In [ ]:
colors = ['absolute zero', 'acid green', 'aero', 'aero blue', 'african violet', 'air superiority blue', 'alabaster', 'alice blue', 'alizarin', 'alloy orange', 'almond', 'amaranth', 'amaranth deep purple', 'amaranth pink', 'amaranth purple', 'amaranth red', 'amazon', 'amber', 'Amber ', 'SAE/ECE', 'amethyst', 'android green', 'antique brass', 'antique bronze', 'antique fuchsia', 'antique ruby', 'antique white', 'Ao ', 'English', 'apple green', 'apricot', 'aqua', 'aquamarine', 'arctic lime', 'army green', 'artichoke', 'arylide yellow', 'ash grey', 'asparagus', 'atomic tangerine', 'auburn', 'aureolin', 'avocado', 'azure', 'Azure ', 'X11/web color', 'baby blue', 'baby blue eyes', 'baby pink', 'baby powder', 'baker-miller pink', 'banana mania', 'banana yellow', 'barbie pink', 'barn red', 'battleship grey', 'beau blue', 'beaver', 'beige', "b'dazzled blue", 'big dip o’ruby', 'big foot feet', 'bisque', 'bistre', 'bistre brown', 'bitter lemon', 'bittersweet', 'bittersweet shimmer', 'black', 'black bean', 'black coral', 'black leather jacket', 'black olive', 'black shadows', 'blanched almond', 'blast-off bronze', 'bleu de france', 'blizzard blue', 'blood red', 'blue', 'Blue ', 'Crayola', 'Blue ', 'Munsell', 'Blue ', 'NCS', 'Blue ', 'Pantone', 'Blue ', 'pigment', 'blue bell', 'blue-gray', 'blue-green', 'blue jeans', 'blue sapphire', 'blue-violet', 'blue yonder', 'blueberry', 'blush', 'bole', 'bone', 'booger buster', 'bottle green', 'brick red', 'bright green', 'bright lilac', 'bright maroon', 'bright navy blue', 'bright pink', 'bright turquoise', 'Bright yellow ', 'Crayola', 'brilliant rose', 'brink pink', 'british racing green', 'bronze', 'brown', 'Brown ', 'web', 'brown sugar', 'brunswick green', 'bud green', 'buff', 'burgundy', 'burlywood', 'burnished brown', 'burnt orange', 'burnt sienna', 'burnt umber', 'byzantine', 'byzantium', 'cadet', 'cadet blue', 'cadet grey', 'cadmium green', 'cadmium orange', 'cadmium red', 'cadmium yellow', 'café au lait', 'café noir', 'calypso', 'cambridge blue', 'camel', 'cameo pink', 'canary yellow', 'candy apple red', 'candy pink', 'caput mortuum', 'cardinal', 'caribbean green', 'carmine', 'carnation pink', 'carnelian', 'carolina blue', 'carrot orange', 'castleton green', 'catawba', 'cedar chest', 'celadon', 'celadon green', 'celeste', 'cerise', 'cerulean', 'cerulean blue', 'cerulean frost', 'Cerulean ', 'Crayola', 'champagne', 'champagne pink', 'charcoal', 'charleston green', 'charm pink', 'Chartreuse ', 'traditional', 'Chartreuse ', 'web', 'cherry blossom pink', 'chestnut', 'china pink', 'china rose', 'chinese red', 'chinese violet', 'Chocolate ', 'traditional', 'Chocolate ', 'web', 'cinereous', 'cinnamon satin', 'citrine', 'citron', 'claret', 'cobalt blue', 'cocoa brown', 'coconut', 'coffee', 'columbia blue', 'congo pink', 'cool grey', 'copper', 'copper penny', 'copper red', 'copper rose', 'coquelicot', 'coral', 'coral pink', 'cordovan', 'cornell red', 'Cornflower blue ', 'web', 'Cornflower blue ', 'Crayola', 'cornsilk', 'cosmic cobalt', 'cosmic latte', 'coyote brown', 'cotton candy', 'cream', 'crimson', 'Crimson ', 'UA', 'cultured', 'cyan', 'cyber grape', 'cyber yellow', 'cyclamen', 'dandelion', 'dark blue', 'dark blue-gray', 'dark brown', 'dark byzantium', 'dark cyan', 'dark electric blue', 'dark fuchsia', 'dark goldenrod', 'Dark gray ', 'X11', 'dark green', 'Dark green ', 'X11', 'dark jungle green', 'dark khaki', 'dark lava', 'dark liver', 'dark magenta', 'dark midnight blue', 'dark moss green', 'dark olive green', 'dark orange', 'dark orchid', 'dark pastel green', 'dark purple', 'dark raspberry', 'dark red', 'dark salmon', 'dark sea green', 'dark sienna', 'dark sky blue', 'dark slate blue', 'dark slate gray', 'dark spring green', 'dark tan', 'dark turquoise', 'dark vanilla', 'dark violet', 'dartmouth green', "davy's grey", 'deep cerise', 'deep champagne', 'deep chestnut', 'deep fuchsia', 'deep jungle green', 'deep lemon', 'deep mauve', 'deep pink', 'deep sky blue', 'deep space sparkle', 'deep taupe', 'denim', 'denim blue', 'desert', 'desert sand', 'diamond', 'dim gray', 'dingy dungeon', 'dirt', 'dodger blue', 'dogwood rose', 'duke blue', 'dutch white', 'earth yellow', 'ebony', 'ecru', 'eerie black', 'eggplant', 'eggshell', 'egyptian blue', 'electric blue', 'electric crimson', 'electric indigo', 'electric lime', 'electric purple', 'electric violet', 'emerald', 'eminence', 'english lavender', 'english red', 'english vermillion', 'english violet', 'eton blue', 'eucalyptus', 'fallow', 'falu red', 'fandango', 'fandango pink', 'fashion fuchsia', 'fawn', 'feldgrau', 'fern green', 'field drab', 'fiery rose', 'firebrick', 'fire engine red', 'fire opal', 'flame', 'flax', 'flirt', 'floral white', 'fluorescent blue', 'Forest green ', 'Crayola', 'Forest green ', 'traditional', 'Forest green ', 'web', 'french beige', 'french bistre', 'french blue', 'french fuchsia', 'french lilac', 'french lime', 'french mauve', 'french pink', 'french raspberry', 'french rose', 'french sky blue', 'french violet', 'frostbite', 'fuchsia', 'Fuchsia ', 'Crayola', 'fuchsia purple', 'fuchsia rose', 'fulvous', 'fuzzy wuzzy', 'gainsboro', 'gamboge', 'garnet', 'generic viridian', 'ghost white', 'glaucous', 'glossy grape', 'go green', 'gold', 'Gold ', 'metallic', 'Gold (web) ', 'Golden', 'Gold ', 'Crayola', 'gold fusion', 'golden brown', 'golden poppy', 'golden yellow', 'goldenrod', 'granite gray', 'granny smith apple', 'Gray ', 'web', 'Gray ', 'X11', 'green', 'Green ', 'Crayola', 'Green ', 'HTML/CSS color', 'Green ', 'Munsell', 'Green ', 'NCS', 'Green ', 'Pantone', 'Green ', 'pigment', 'Green ', 'RYB', 'green-blue', 'Green-blue ', 'Crayola', 'green-cyan', 'green lizard', 'green sheen', 'green-yellow', 'Green-yellow ', 'Crayola', 'grullo', 'gunmetal', 'han blue', 'han purple', 'hansa yellow', 'harlequin', 'harvest gold', 'heat wave', 'heliotrope', 'heliotrope gray', 'hollywood cerise', 'honeydew', 'honolulu blue', "hooker's green", 'hot fuchsia', 'hot magenta', 'hot pink', 'hunter green', 'iceberg', 'icterine', 'illuminating emerald', 'imperial red', 'inchworm', 'independence', 'india green', 'indian red', 'indian yellow', 'indigo', 'indigo dye', 'International orange ', 'aerospace', 'International orange ', 'engineering', 'International orange ', 'Golden Gate Bridge', 'iris', 'irresistible', 'isabelline', 'italian sky blue', 'ivory', 'jade', 'japanese carmine', 'japanese violet', 'jasmine', 'jazzberry jam', 'jet', 'jonquil', 'june bud', 'jungle green', 'kelly green', 'keppel', 'key lime', 'Khaki (web) ', 'Khaki', 'Khaki (X11) ', 'Light khaki', 'kobe', 'kobi', 'kobicha', 'kombu green', 'ksu purple', 'la salle green', 'languid lavender', 'lanzones', 'lapis lazuli', 'laser lemon', 'laurel green', 'lava', 'Lavender ', 'floral', 'Lavender ', 'web', 'lavender blue', 'lavender blush', 'lavender gray', 'lavender indigo', 'lavender magenta', 'lavender mist', 'lavender pink', 'lavender purple', 'lavender rose', 'lawn green', 'lemon', 'lemon chiffon', 'lemon curry', 'lemon glacier', 'lemon iced tea', 'lemon lime', 'lemon lime', 'lemon meringue', 'lemon yellow', 'Lemon yellow ', 'Crayola', 'lenurple', 'liberty', 'licorice', 'light apricot', 'light blue', 'light brown', 'light carmine pink', 'light chocolate cosmos', 'light cobalt blue', 'light coral', 'light cornflower blue', 'light crimson', 'light cyan', 'light deep pink', 'light french beige', 'light fuchsia pink', 'light gold', 'light goldenrod', 'light goldenrod yellow', 'light gray', 'light grayish magenta', 'light green', 'light hot pink', 'light khaki', 'light medium orchid', 'light moss green', 'light mustard', 'light orange', 'light orchid', 'light pastel purple', 'light periwinkle', 'light pink', 'light red', 'light red ochre', 'light salmon', 'light salmon pink', 'light sea green', 'light silver', 'light sky blue', 'light slate gray', 'light steel blue', 'light taupe', 'light thulian pink', 'light turquoise', 'light violet', 'light yellow', 'lilac', 'lilac luster', 'Lime ', 'color wheel', 'Lime (web) ', 'X11 green', 'lime green', 'limerick', 'lincoln green', 'linen', 'lion', 'liseran purple', 'little boy blue', 'little girl pink', 'liver', 'Liver ', 'dogs', 'Liver ', 'organ', 'liver chestnut', 'livid', 'lotion', 'lotion blue', 'lotion pink', 'lumber', 'lust', 'maastricht blue', 'macaroni and cheese', 'madder lake', 'magenta', 'Magenta ', 'Pantone', 'mahogany', 'maize', 'Maize ', 'Crayola', 'majorelle blue', 'malachite', 'manatee', 'mandarin', 'mango', 'mango green', 'mango tango', 'mantis', 'mardi gras', 'marigold', 'Maroon ', 'Crayola', 'Maroon ', 'HTML/CSS', 'Maroon ', 'X11', 'mauve', 'mauve taupe', 'mauvelous', 'maximum blue', 'maximum blue green', 'maximum blue purple', 'maximum green', 'maximum green yellow', 'maximum orange', 'maximum purple', 'maximum pink', 'maximum red', 'maximum red purple', 'maximum violet', 'maximum yellow', 'maximum yellow red', 'may green', 'maya blue', 'meat brown', 'medium aquamarine', 'medium blue', 'medium candy apple red', 'medium carmine', 'medium champagne', 'medium electric blue', 'medium green', 'medium jungle green', 'medium lavender magenta', 'medium orange', 'medium orchid', 'medium persian blue', 'medium pink', 'medium purple', 'medium red', 'medium red-violet', 'medium ruby', 'medium sea green', 'medium sky blue', 'medium slate blue', 'medium spring bud', 'medium spring green', 'medium taupe', 'medium turquoise', 'medium tuscan red', 'medium vermilion', 'medium violet', 'medium violet-red', 'medium yellow', 'mellow apricot', 'mellow yellow', 'melon', 'Melon ', 'Crayola', 'menthol', 'metallic blue', 'metallic bronze', 'metallic brown', 'metallic gold', 'metallic green', 'metallic orange', 'metallic pink', 'metallic red', 'metallic seaweed', 'metallic silver', 'metallic sunburst', 'metallic violet', 'metallic yellow', 'mexican pink', 'microsoft blue', 'microsoft edge blue', 'microsoft green', 'microsoft red', 'microsoft yellow', 'middle blue', 'middle blue green', 'middle blue purple', 'middle grey', 'middle green', 'middle green yellow', 'middle purple', 'middle red', 'middle red purple', 'middle yellow', 'middle yellow red', 'midnight', 'midnight blue', 'midnight blue', 'Midnight green ', 'eagle green', 'mikado yellow', 'milk', 'milk chocolate', 'mimi pink', 'mindaro', 'ming', 'minion yellow', 'mint', 'mint cream', 'mint green', 'misty moss', 'misty rose', 'moccasin', 'mocha', 'mode beige', 'moonstone', 'moonstone blue', 'mordant red 19', 'morning blue', 'moss green', 'mountain meadow', 'mountbatten pink', 'msu green', 'mud', 'mughal green', 'mulberry', 'Mulberry ', 'Crayola', "mummy's tomb", 'mustard', 'mustard brown', 'mustard green', 'mustard yellow', 'myrtle green', 'mystic', 'mystic maroon', 'mystic red', 'nadeshiko pink', 'napier green', 'naples yellow', 'navajo white', 'navy blue', 'Navy blue ', 'Crayola', 'navy purple', 'neon blue', 'neon brown', 'neon carrot', 'neon cyan', 'neon fuchsia', 'neon gold', 'neon gray', 'neon dark green', 'neon green', 'neon green', 'neon pink', 'neon purple', 'neon red', 'neon scarlet', 'neon silver', 'neon tangerine', 'neon yellow', 'new car', 'new york pink', 'nickel', 'nintendo red', 'non-photo blue', 'nyanza', 'ocean blue', 'ocean boat blue', 'ocean green', 'ochre', 'office green', 'ogre odor', 'old burgundy', 'old gold', 'old heliotrope', 'old lace', 'old lavender', 'old mauve', 'old moss green', 'old rose', 'old silver', 'olive', 'Olive drab ', '#3', 'olive drab #7', 'olive green', 'olivine', 'onyx', 'opal', 'opera mauve', 'orange', 'Orange ', 'color wheel', 'Orange ', 'Crayola', 'Orange ', 'Pantone', 'Orange ', 'RYB', 'Orange ', 'web', 'orange iced tea', 'orange peel', 'orange-red', 'Orange-red ', 'Crayola', 'orange soda', 'orange soda', 'orange-yellow', 'Orange-yellow ', 'Crayola', 'orchid', 'orchid pink', 'Orchid ', 'Crayola', 'orioles orange', 'otter brown', 'outer space', 'Outer space ', 'Crayola', 'outrageous orange', 'oxblood', 'oxford blue', 'oxley', 'ou crimson red', 'pacific blue', 'pakistan green', 'palatinate blue', 'palatinate purple', 'pale aqua', 'pale blue', 'pale brown', 'pale carmine', 'pale cerulean', 'pale chestnut', 'pale copper', 'pale cornflower blue', 'pale cyan', 'pale gold', 'pale goldenrod', 'pale green', 'pale lavender', 'pale magenta', 'pale magenta-pink', 'pale pink', 'pale plum', 'pale red-violet', 'pale robin egg blue', 'pale silver', 'pale spring bud', 'pale taupe', 'pale turquoise', 'pale violet', 'pale violet-red', 'palm leaf', 'pansy purple', 'paolo veronese green', 'papaya whip', 'paradise pink', 'parchment', 'paris green', 'parrot pink', 'pastel blue', 'pastel brown', 'pastel gray', 'pastel green', 'pastel magenta', 'pastel orange', 'pastel pink', 'pastel purple', 'pastel red', 'pastel violet', 'pastel yellow', 'patriarch', "payne's grey", 'peach', 'Peach ', 'Crayola', 'peach-orange', 'peach puff', 'peach-yellow', 'pear', 'pearl', 'pearl aqua', 'pearly purple', 'peridot', 'periwinkle', 'Periwinkle ', 'Crayola', 'permanent geranium lake', 'persian blue', 'persian green', 'persian indigo', 'persian orange', 'persian pink', 'persian plum', 'persian red', 'persian rose', 'persimmon', 'peru', 'petal', 'pewter blue', 'philippine blue', 'philippine bronze', 'philippine brown', 'philippine gold', 'philippine golden yellow', 'philippine gray', 'philippine green', 'philippine indigo', 'philippine orange', 'philippine pink', 'philippine red', 'philippine silver', 'philippine sky blue', 'philippine violet', 'philippine yellow', 'phlox', 'phthalo blue', 'phthalo green', 'picton blue', 'pictorial carmine', 'piggy pink', 'pine green', 'pine tree', 'pineapple', 'pink', 'Pink ', 'Pantone', 'Pink Diamond ', 'Ace Hardware Color', 'Pink Diamond ', 'Independent Retailers Colors', 'pink flamingo', 'pink lace', 'pink lavender', 'pink-orange', 'pink pearl', 'pink raspberry', 'pink sherbet', 'pistachio', 'pixie powder', 'platinum', 'plum', 'Plum ', 'web', 'plump purple', 'poison purple', 'police blue', 'polished pine', 'pomp and power', 'popstar', 'portland orange', 'powder blue', 'prilly blue', 'prilly pink', 'prilly red', 'princess perfume', 'princeton orange', 'prune', 'prussian blue', 'psychedelic purple', 'puce', 'puce red', 'Pullman Brown ', 'UPS Brown', 'pullman green', 'pumpkin', 'Purple ', 'HTML', 'Purple ', 'Munsell', 'Purple ', 'X11', 'purple heart', 'purple mountain majesty', 'purple navy', 'purple pizzazz', 'purple plum', 'purple taupe', 'purpureus', 'quartz', 'queen blue', 'queen pink', 'quick silver', 'quinacridone magenta', 'quincy', 'rackley', 'radical red', 'raisin black', 'rajah', 'raspberry', 'raspberry glace', 'raspberry pink', 'raspberry rose', 'raw sienna', 'raw umber', 'razzle dazzle rose', 'razzmatazz', 'razzmic berry', 'rebecca purple', 'red', 'Red ', 'Crayola', 'Red ', 'Munsell', 'Red ', 'NCS', 'Red ', 'Pantone', 'Red ', 'pigment', 'Red ', 'RYB', 'red-brown', 'red cola', 'red devil', 'red-orange', 'Red-orange ', 'Crayola', 'Red-orange ', 'Color wheel', 'red-purple', 'red rum', 'red salsa', 'red strawberry', 'red-violet', 'Red-violet ', 'Crayola', 'Red-violet ', 'Color wheel', 'redwood', 'registration black', 'resolution blue', 'rhythm', 'rich brilliant lavender', 'rich carmine', 'rich electric blue', 'rich lavender', 'rich lilac', 'rich maroon', 'rifle green', 'ripe mango', 'roast coffee', 'robin egg blue', 'rocket metallic', 'roman silver', 'root beer', 'rose', 'rose bonbon', 'rose dust', 'rose ebony', 'rose garnet', 'rose gold', 'rose madder', 'rose pink', 'rose quartz', 'rose quartz pink', 'rose red', 'rose taupe', 'rose vale', 'rosewood', 'rosy brown', 'royal azure', 'royal blue', 'royal blue', 'royal brown', 'royal fuchsia', 'royal green', 'royal orange', 'royal pink', 'royal red', 'royal red', 'royal purple', 'royal yellow', 'ruber', 'rubine red', 'ruby', 'ruby red', 'rufous', 'rum', 'russet', 'russian green', 'russian violet', 'rust', 'rusty red', 'sacramento state green', 'saddle brown', 'safety orange', 'Safety orange ', 'blaze orange', 'safety yellow', 'saffron', 'sage', "st. patrick's blue", 'salem', 'salmon', 'salmon rose', 'salmon pink', 'samsung blue', 'sand', 'sand dune', 'sandstorm', 'sandy brown', 'sandy tan', 'sandy taupe', 'sap green', 'sapphire', 'sapphire blue', 'sasquatch socks', 'satin sheen gold', 'scarlet', 'Scarlet ', 'Crayola', 'schauss pink', 'school bus yellow', "screamin' green", 'sea blue', 'sea foam green', 'sea green', 'Sea green ', 'Crayola', 'sea serpent', 'seal brown', 'seashell', 'selective yellow', 'sepia', 'shadow', 'shadow blue', 'shampoo', 'shamrock green', 'shandy', 'sheen green', 'shimmering blush', 'shiny shamrock', 'shocking pink', 'Shocking pink ', 'Crayola', 'sienna', 'silver', 'Silver ', 'Crayola', 'Silver ', 'Metallic', 'silver chalice', 'silver foil', 'silver lake blue', 'silver pink', 'silver sand', 'sinopia', 'sizzling red', 'sizzling sunrise', 'skobeloff', 'sky blue', 'Sky blue ', 'Crayola', 'sky magenta', 'slate blue', 'slate gray', 'slimy green', 'Smalt ', 'Dark powder blue', 'smashed pumpkin', 'smitten', 'smoke', 'smokey topaz', 'smoky black', 'snow', 'soap', 'solid pink', 'sonic silver', 'spartan crimson', 'space cadet', 'spanish bistre', 'spanish blue', 'spanish carmine', 'spanish crimson', 'spanish gray', 'spanish green', 'spanish orange', 'spanish pink', 'spanish purple', 'spanish red', 'spanish sky blue', 'spanish violet', 'spanish viridian', 'spanish yellow', 'spicy mix', 'spiro disco ball', 'spring bud', 'spring frost', 'spring green', 'Spring green ', 'Crayola', 'star command blue', 'steel blue', 'steel pink', 'steel teal', 'stil de grain yellow', 'straw', 'strawberry', 'stop red', 'strawberry iced tea', 'strawberry red', 'sugar plum', 'sunburnt cyclops', 'sunglow', 'sunny', 'sunray', 'sunset', 'sunset orange', 'super pink', 'sweet brown', 'taffy', 'tan', 'Tan ', 'Crayola', 'tangelo', 'tangerine', 'tangerine yellow', 'tango pink', 'tart orange', 'taupe', 'taupe gray', 'tea green', 'tea rose', 'tea rose', 'teal', 'teal blue', 'teal deer', 'teal green', 'telemagenta', 'temptress', 'Tenné ', 'tawny', 'terra cotta', 'thistle', 'Thistle ', 'Crayola', 'thulian pink', 'tickle me pink', 'tiffany blue', "tiger's eye", 'timberwolf', 'titanium', 'titanium yellow', 'tomato', 'tomato sauce', 'toolbox', 'tooth', 'topaz', 'tractor red', 'trolley grey', 'tropical rain forest', 'tropical violet', 'true blue', 'tufts blue', 'tulip', 'tumbleweed', 'turkish rose', 'turquoise', 'turquoise blue', 'turquoise green', 'turquoise surf', 'turtle green', 'tuscan', 'tuscan brown', 'tuscan red', 'tuscan tan', 'tuscany', 'twilight lavender', 'twitter blue', 'tyrian purple', 'ube', 'ultramarine', 'ultramarine blue', 'Ultramarine blue ', "Caran d'Ache", 'ultra pink', 'ultra red', 'umber', 'unbleached silk', 'united nations blue', 'unmellow yellow', 'up maroon', 'upsdell red', 'urobilin', 'vampire black', 'van dyke brown', 'vanilla', 'vanilla ice', 'vegas gold', 'venetian red', 'verdigris', 'vermilion', 'vermilion', 'veronica', 'verse green', 'very light azure', 'very light blue', 'very light malachite green', 'very light tangelo', 'very pale orange', 'very pale yellow', 'vine green', 'violet', 'Violet ', "Caran d'Ache", 'Violet ', 'color wheel', 'Violet ', 'crayola', 'Violet ', 'RYB', 'Violet ', 'web', 'violet-blue', 'Violet-blue ', 'Crayola', 'violet-red', 'violin brown', 'viridian', 'viridian green', 'vista blue', 'vivid amber', 'vivid auburn', 'vivid burgundy', 'vivid cerise', 'vivid cerulean', 'vivid crimson', 'vivid gamboge', 'vivid lime green', 'vivid malachite', 'vivid mulberry', 'vivid orange', 'vivid orange peel', 'vivid orchid', 'vivid raspberry', 'vivid red', 'vivid red-tangelo', 'vivid sky blue', 'vivid tangelo', 'vivid tangerine', 'vivid vermilion', 'vivid violet', 'vivid yellow', 'water', 'watermelon', 'watermelon red', 'watermelon yellow', 'waterspout', 'weldon blue', 'wenge', 'wheat', 'white', 'white chocolate', 'white coffee', 'white smoke', 'wild orchid', 'wild strawberry', 'wild watermelon', 'willpower orange', 'windsor tan', 'wine', 'wine dregs', 'wine red', 'winter sky', 'winter wizard', 'wintergreen dream', 'wisteria', 'wood brown', 'xanadu', 'yellow', 'Yellow ', 'Crayola', 'Yellow ', 'Munsell', 'Yellow ', 'NCS', 'Yellow ', 'Pantone', 'Yellow ', 'process', 'Yellow ', 'RYB', 'yellow-green', 'Yellow-green ', 'Crayola', 'yellow orange', 'Yellow Orange ', 'Color Wheel', 'yellow rose', 'yellow sunshine', 'yinmn blue', 'zaffre', 'zebra white', 'zinnwaldite', 'zomp']
#binary: yes/no
#numbers
def is_number(s):
  if s.isnumeric() or s.isdigit():
    return True
  try:
      float(s)
      return True
  except ValueError:
      return False
#unanswerable: ["unanswerable",'unsuitable','unsuitable image']
# label2questype = []
binary_labels = []
unanswerable_labels = []
number_labels = []
color_labels = []
other_labels = []

import json
with open('data/vqa/trainval_label2ans.json') as f1:
  label2ans = json.load(f1)

for label,ans in enumerate(label2ans):
  if ans in ['yes','no']:
    binary_labels.append(label)
    # label2questype.append("binary")
    continue
  if ans in ["unanswerable",'unsuitable','unsuitable image']:
    unanswerable_labels.append(label)
    # label2questype.append("unanswerable")
    continue
  if is_number(ans):
    number_labels.append(label)
    # label2questype.append("number")
    continue
  if ans in colors:
    color_labels.append(label)
    # label2questype.append("color")
    continue
  other_labels.append(label)
  # label2questype.append("other")


quesType2labels = {}
quesType2labels["binary"] = binary_labels
quesType2labels["unanswerable"] = unanswerable_labels
quesType2labels["number"] = number_labels
quesType2labels["color"] = color_labels
quesType2labels["other"] = other_labels


with open('data/vqa/trainval_questype2labels.json', 'w') as outfile:
    json.dump(quesType2labels, outfile)

In [ ]:
with open('data/vqa/valid.json') as f1:
  val_info = json.load(f1)

with open('data/vqa/train.json') as f1:
  train_info = json.load(f1)

def label2questype(label):
  if label in ['yes','no']:
    return "binary"
  if label in ["unanswerable",'unsuitable','unsuitable image']:
    return "unanswerable"
  if is_number(label):
    return "number"
  if label in colors:
    return "color"
  return "other"

colors = ['absolute zero', 'acid green', 'aero', 'aero blue', 'african violet', 'air superiority blue', 'alabaster', 'alice blue', 'alizarin', 'alloy orange', 'almond', 'amaranth', 'amaranth deep purple', 'amaranth pink', 'amaranth purple', 'amaranth red', 'amazon', 'amber', 'Amber ', 'SAE/ECE', 'amethyst', 'android green', 'antique brass', 'antique bronze', 'antique fuchsia', 'antique ruby', 'antique white', 'Ao ', 'English', 'apple green', 'apricot', 'aqua', 'aquamarine', 'arctic lime', 'army green', 'artichoke', 'arylide yellow', 'ash grey', 'asparagus', 'atomic tangerine', 'auburn', 'aureolin', 'avocado', 'azure', 'Azure ', 'X11/web color', 'baby blue', 'baby blue eyes', 'baby pink', 'baby powder', 'baker-miller pink', 'banana mania', 'banana yellow', 'barbie pink', 'barn red', 'battleship grey', 'beau blue', 'beaver', 'beige', "b'dazzled blue", 'big dip o’ruby', 'big foot feet', 'bisque', 'bistre', 'bistre brown', 'bitter lemon', 'bittersweet', 'bittersweet shimmer', 'black', 'black bean', 'black coral', 'black leather jacket', 'black olive', 'black shadows', 'blanched almond', 'blast-off bronze', 'bleu de france', 'blizzard blue', 'blood red', 'blue', 'Blue ', 'Crayola', 'Blue ', 'Munsell', 'Blue ', 'NCS', 'Blue ', 'Pantone', 'Blue ', 'pigment', 'blue bell', 'blue-gray', 'blue-green', 'blue jeans', 'blue sapphire', 'blue-violet', 'blue yonder', 'blueberry', 'blush', 'bole', 'bone', 'booger buster', 'bottle green', 'brick red', 'bright green', 'bright lilac', 'bright maroon', 'bright navy blue', 'bright pink', 'bright turquoise', 'Bright yellow ', 'Crayola', 'brilliant rose', 'brink pink', 'british racing green', 'bronze', 'brown', 'Brown ', 'web', 'brown sugar', 'brunswick green', 'bud green', 'buff', 'burgundy', 'burlywood', 'burnished brown', 'burnt orange', 'burnt sienna', 'burnt umber', 'byzantine', 'byzantium', 'cadet', 'cadet blue', 'cadet grey', 'cadmium green', 'cadmium orange', 'cadmium red', 'cadmium yellow', 'café au lait', 'café noir', 'calypso', 'cambridge blue', 'camel', 'cameo pink', 'canary yellow', 'candy apple red', 'candy pink', 'caput mortuum', 'cardinal', 'caribbean green', 'carmine', 'carnation pink', 'carnelian', 'carolina blue', 'carrot orange', 'castleton green', 'catawba', 'cedar chest', 'celadon', 'celadon green', 'celeste', 'cerise', 'cerulean', 'cerulean blue', 'cerulean frost', 'Cerulean ', 'Crayola', 'champagne', 'champagne pink', 'charcoal', 'charleston green', 'charm pink', 'Chartreuse ', 'traditional', 'Chartreuse ', 'web', 'cherry blossom pink', 'chestnut', 'china pink', 'china rose', 'chinese red', 'chinese violet', 'Chocolate ', 'traditional', 'Chocolate ', 'web', 'cinereous', 'cinnamon satin', 'citrine', 'citron', 'claret', 'cobalt blue', 'cocoa brown', 'coconut', 'coffee', 'columbia blue', 'congo pink', 'cool grey', 'copper', 'copper penny', 'copper red', 'copper rose', 'coquelicot', 'coral', 'coral pink', 'cordovan', 'cornell red', 'Cornflower blue ', 'web', 'Cornflower blue ', 'Crayola', 'cornsilk', 'cosmic cobalt', 'cosmic latte', 'coyote brown', 'cotton candy', 'cream', 'crimson', 'Crimson ', 'UA', 'cultured', 'cyan', 'cyber grape', 'cyber yellow', 'cyclamen', 'dandelion', 'dark blue', 'dark blue-gray', 'dark brown', 'dark byzantium', 'dark cyan', 'dark electric blue', 'dark fuchsia', 'dark goldenrod', 'Dark gray ', 'X11', 'dark green', 'Dark green ', 'X11', 'dark jungle green', 'dark khaki', 'dark lava', 'dark liver', 'dark magenta', 'dark midnight blue', 'dark moss green', 'dark olive green', 'dark orange', 'dark orchid', 'dark pastel green', 'dark purple', 'dark raspberry', 'dark red', 'dark salmon', 'dark sea green', 'dark sienna', 'dark sky blue', 'dark slate blue', 'dark slate gray', 'dark spring green', 'dark tan', 'dark turquoise', 'dark vanilla', 'dark violet', 'dartmouth green', "davy's grey", 'deep cerise', 'deep champagne', 'deep chestnut', 'deep fuchsia', 'deep jungle green', 'deep lemon', 'deep mauve', 'deep pink', 'deep sky blue', 'deep space sparkle', 'deep taupe', 'denim', 'denim blue', 'desert', 'desert sand', 'diamond', 'dim gray', 'dingy dungeon', 'dirt', 'dodger blue', 'dogwood rose', 'duke blue', 'dutch white', 'earth yellow', 'ebony', 'ecru', 'eerie black', 'eggplant', 'eggshell', 'egyptian blue', 'electric blue', 'electric crimson', 'electric indigo', 'electric lime', 'electric purple', 'electric violet', 'emerald', 'eminence', 'english lavender', 'english red', 'english vermillion', 'english violet', 'eton blue', 'eucalyptus', 'fallow', 'falu red', 'fandango', 'fandango pink', 'fashion fuchsia', 'fawn', 'feldgrau', 'fern green', 'field drab', 'fiery rose', 'firebrick', 'fire engine red', 'fire opal', 'flame', 'flax', 'flirt', 'floral white', 'fluorescent blue', 'Forest green ', 'Crayola', 'Forest green ', 'traditional', 'Forest green ', 'web', 'french beige', 'french bistre', 'french blue', 'french fuchsia', 'french lilac', 'french lime', 'french mauve', 'french pink', 'french raspberry', 'french rose', 'french sky blue', 'french violet', 'frostbite', 'fuchsia', 'Fuchsia ', 'Crayola', 'fuchsia purple', 'fuchsia rose', 'fulvous', 'fuzzy wuzzy', 'gainsboro', 'gamboge', 'garnet', 'generic viridian', 'ghost white', 'glaucous', 'glossy grape', 'go green', 'gold', 'Gold ', 'metallic', 'Gold (web) ', 'Golden', 'Gold ', 'Crayola', 'gold fusion', 'golden brown', 'golden poppy', 'golden yellow', 'goldenrod', 'granite gray', 'granny smith apple', 'Gray ', 'web', 'Gray ', 'X11', 'green', 'Green ', 'Crayola', 'Green ', 'HTML/CSS color', 'Green ', 'Munsell', 'Green ', 'NCS', 'Green ', 'Pantone', 'Green ', 'pigment', 'Green ', 'RYB', 'green-blue', 'Green-blue ', 'Crayola', 'green-cyan', 'green lizard', 'green sheen', 'green-yellow', 'Green-yellow ', 'Crayola', 'grullo', 'gunmetal', 'han blue', 'han purple', 'hansa yellow', 'harlequin', 'harvest gold', 'heat wave', 'heliotrope', 'heliotrope gray', 'hollywood cerise', 'honeydew', 'honolulu blue', "hooker's green", 'hot fuchsia', 'hot magenta', 'hot pink', 'hunter green', 'iceberg', 'icterine', 'illuminating emerald', 'imperial red', 'inchworm', 'independence', 'india green', 'indian red', 'indian yellow', 'indigo', 'indigo dye', 'International orange ', 'aerospace', 'International orange ', 'engineering', 'International orange ', 'Golden Gate Bridge', 'iris', 'irresistible', 'isabelline', 'italian sky blue', 'ivory', 'jade', 'japanese carmine', 'japanese violet', 'jasmine', 'jazzberry jam', 'jet', 'jonquil', 'june bud', 'jungle green', 'kelly green', 'keppel', 'key lime', 'Khaki (web) ', 'Khaki', 'Khaki (X11) ', 'Light khaki', 'kobe', 'kobi', 'kobicha', 'kombu green', 'ksu purple', 'la salle green', 'languid lavender', 'lanzones', 'lapis lazuli', 'laser lemon', 'laurel green', 'lava', 'Lavender ', 'floral', 'Lavender ', 'web', 'lavender blue', 'lavender blush', 'lavender gray', 'lavender indigo', 'lavender magenta', 'lavender mist', 'lavender pink', 'lavender purple', 'lavender rose', 'lawn green', 'lemon', 'lemon chiffon', 'lemon curry', 'lemon glacier', 'lemon iced tea', 'lemon lime', 'lemon lime', 'lemon meringue', 'lemon yellow', 'Lemon yellow ', 'Crayola', 'lenurple', 'liberty', 'licorice', 'light apricot', 'light blue', 'light brown', 'light carmine pink', 'light chocolate cosmos', 'light cobalt blue', 'light coral', 'light cornflower blue', 'light crimson', 'light cyan', 'light deep pink', 'light french beige', 'light fuchsia pink', 'light gold', 'light goldenrod', 'light goldenrod yellow', 'light gray', 'light grayish magenta', 'light green', 'light hot pink', 'light khaki', 'light medium orchid', 'light moss green', 'light mustard', 'light orange', 'light orchid', 'light pastel purple', 'light periwinkle', 'light pink', 'light red', 'light red ochre', 'light salmon', 'light salmon pink', 'light sea green', 'light silver', 'light sky blue', 'light slate gray', 'light steel blue', 'light taupe', 'light thulian pink', 'light turquoise', 'light violet', 'light yellow', 'lilac', 'lilac luster', 'Lime ', 'color wheel', 'Lime (web) ', 'X11 green', 'lime green', 'limerick', 'lincoln green', 'linen', 'lion', 'liseran purple', 'little boy blue', 'little girl pink', 'liver', 'Liver ', 'dogs', 'Liver ', 'organ', 'liver chestnut', 'livid', 'lotion', 'lotion blue', 'lotion pink', 'lumber', 'lust', 'maastricht blue', 'macaroni and cheese', 'madder lake', 'magenta', 'Magenta ', 'Pantone', 'mahogany', 'maize', 'Maize ', 'Crayola', 'majorelle blue', 'malachite', 'manatee', 'mandarin', 'mango', 'mango green', 'mango tango', 'mantis', 'mardi gras', 'marigold', 'Maroon ', 'Crayola', 'Maroon ', 'HTML/CSS', 'Maroon ', 'X11', 'mauve', 'mauve taupe', 'mauvelous', 'maximum blue', 'maximum blue green', 'maximum blue purple', 'maximum green', 'maximum green yellow', 'maximum orange', 'maximum purple', 'maximum pink', 'maximum red', 'maximum red purple', 'maximum violet', 'maximum yellow', 'maximum yellow red', 'may green', 'maya blue', 'meat brown', 'medium aquamarine', 'medium blue', 'medium candy apple red', 'medium carmine', 'medium champagne', 'medium electric blue', 'medium green', 'medium jungle green', 'medium lavender magenta', 'medium orange', 'medium orchid', 'medium persian blue', 'medium pink', 'medium purple', 'medium red', 'medium red-violet', 'medium ruby', 'medium sea green', 'medium sky blue', 'medium slate blue', 'medium spring bud', 'medium spring green', 'medium taupe', 'medium turquoise', 'medium tuscan red', 'medium vermilion', 'medium violet', 'medium violet-red', 'medium yellow', 'mellow apricot', 'mellow yellow', 'melon', 'Melon ', 'Crayola', 'menthol', 'metallic blue', 'metallic bronze', 'metallic brown', 'metallic gold', 'metallic green', 'metallic orange', 'metallic pink', 'metallic red', 'metallic seaweed', 'metallic silver', 'metallic sunburst', 'metallic violet', 'metallic yellow', 'mexican pink', 'microsoft blue', 'microsoft edge blue', 'microsoft green', 'microsoft red', 'microsoft yellow', 'middle blue', 'middle blue green', 'middle blue purple', 'middle grey', 'middle green', 'middle green yellow', 'middle purple', 'middle red', 'middle red purple', 'middle yellow', 'middle yellow red', 'midnight', 'midnight blue', 'midnight blue', 'Midnight green ', 'eagle green', 'mikado yellow', 'milk', 'milk chocolate', 'mimi pink', 'mindaro', 'ming', 'minion yellow', 'mint', 'mint cream', 'mint green', 'misty moss', 'misty rose', 'moccasin', 'mocha', 'mode beige', 'moonstone', 'moonstone blue', 'mordant red 19', 'morning blue', 'moss green', 'mountain meadow', 'mountbatten pink', 'msu green', 'mud', 'mughal green', 'mulberry', 'Mulberry ', 'Crayola', "mummy's tomb", 'mustard', 'mustard brown', 'mustard green', 'mustard yellow', 'myrtle green', 'mystic', 'mystic maroon', 'mystic red', 'nadeshiko pink', 'napier green', 'naples yellow', 'navajo white', 'navy blue', 'Navy blue ', 'Crayola', 'navy purple', 'neon blue', 'neon brown', 'neon carrot', 'neon cyan', 'neon fuchsia', 'neon gold', 'neon gray', 'neon dark green', 'neon green', 'neon green', 'neon pink', 'neon purple', 'neon red', 'neon scarlet', 'neon silver', 'neon tangerine', 'neon yellow', 'new car', 'new york pink', 'nickel', 'nintendo red', 'non-photo blue', 'nyanza', 'ocean blue', 'ocean boat blue', 'ocean green', 'ochre', 'office green', 'ogre odor', 'old burgundy', 'old gold', 'old heliotrope', 'old lace', 'old lavender', 'old mauve', 'old moss green', 'old rose', 'old silver', 'olive', 'Olive drab ', '#3', 'olive drab #7', 'olive green', 'olivine', 'onyx', 'opal', 'opera mauve', 'orange', 'Orange ', 'color wheel', 'Orange ', 'Crayola', 'Orange ', 'Pantone', 'Orange ', 'RYB', 'Orange ', 'web', 'orange iced tea', 'orange peel', 'orange-red', 'Orange-red ', 'Crayola', 'orange soda', 'orange soda', 'orange-yellow', 'Orange-yellow ', 'Crayola', 'orchid', 'orchid pink', 'Orchid ', 'Crayola', 'orioles orange', 'otter brown', 'outer space', 'Outer space ', 'Crayola', 'outrageous orange', 'oxblood', 'oxford blue', 'oxley', 'ou crimson red', 'pacific blue', 'pakistan green', 'palatinate blue', 'palatinate purple', 'pale aqua', 'pale blue', 'pale brown', 'pale carmine', 'pale cerulean', 'pale chestnut', 'pale copper', 'pale cornflower blue', 'pale cyan', 'pale gold', 'pale goldenrod', 'pale green', 'pale lavender', 'pale magenta', 'pale magenta-pink', 'pale pink', 'pale plum', 'pale red-violet', 'pale robin egg blue', 'pale silver', 'pale spring bud', 'pale taupe', 'pale turquoise', 'pale violet', 'pale violet-red', 'palm leaf', 'pansy purple', 'paolo veronese green', 'papaya whip', 'paradise pink', 'parchment', 'paris green', 'parrot pink', 'pastel blue', 'pastel brown', 'pastel gray', 'pastel green', 'pastel magenta', 'pastel orange', 'pastel pink', 'pastel purple', 'pastel red', 'pastel violet', 'pastel yellow', 'patriarch', "payne's grey", 'peach', 'Peach ', 'Crayola', 'peach-orange', 'peach puff', 'peach-yellow', 'pear', 'pearl', 'pearl aqua', 'pearly purple', 'peridot', 'periwinkle', 'Periwinkle ', 'Crayola', 'permanent geranium lake', 'persian blue', 'persian green', 'persian indigo', 'persian orange', 'persian pink', 'persian plum', 'persian red', 'persian rose', 'persimmon', 'peru', 'petal', 'pewter blue', 'philippine blue', 'philippine bronze', 'philippine brown', 'philippine gold', 'philippine golden yellow', 'philippine gray', 'philippine green', 'philippine indigo', 'philippine orange', 'philippine pink', 'philippine red', 'philippine silver', 'philippine sky blue', 'philippine violet', 'philippine yellow', 'phlox', 'phthalo blue', 'phthalo green', 'picton blue', 'pictorial carmine', 'piggy pink', 'pine green', 'pine tree', 'pineapple', 'pink', 'Pink ', 'Pantone', 'Pink Diamond ', 'Ace Hardware Color', 'Pink Diamond ', 'Independent Retailers Colors', 'pink flamingo', 'pink lace', 'pink lavender', 'pink-orange', 'pink pearl', 'pink raspberry', 'pink sherbet', 'pistachio', 'pixie powder', 'platinum', 'plum', 'Plum ', 'web', 'plump purple', 'poison purple', 'police blue', 'polished pine', 'pomp and power', 'popstar', 'portland orange', 'powder blue', 'prilly blue', 'prilly pink', 'prilly red', 'princess perfume', 'princeton orange', 'prune', 'prussian blue', 'psychedelic purple', 'puce', 'puce red', 'Pullman Brown ', 'UPS Brown', 'pullman green', 'pumpkin', 'Purple ', 'HTML', 'Purple ', 'Munsell', 'Purple ', 'X11', 'purple heart', 'purple mountain majesty', 'purple navy', 'purple pizzazz', 'purple plum', 'purple taupe', 'purpureus', 'quartz', 'queen blue', 'queen pink', 'quick silver', 'quinacridone magenta', 'quincy', 'rackley', 'radical red', 'raisin black', 'rajah', 'raspberry', 'raspberry glace', 'raspberry pink', 'raspberry rose', 'raw sienna', 'raw umber', 'razzle dazzle rose', 'razzmatazz', 'razzmic berry', 'rebecca purple', 'red', 'Red ', 'Crayola', 'Red ', 'Munsell', 'Red ', 'NCS', 'Red ', 'Pantone', 'Red ', 'pigment', 'Red ', 'RYB', 'red-brown', 'red cola', 'red devil', 'red-orange', 'Red-orange ', 'Crayola', 'Red-orange ', 'Color wheel', 'red-purple', 'red rum', 'red salsa', 'red strawberry', 'red-violet', 'Red-violet ', 'Crayola', 'Red-violet ', 'Color wheel', 'redwood', 'registration black', 'resolution blue', 'rhythm', 'rich brilliant lavender', 'rich carmine', 'rich electric blue', 'rich lavender', 'rich lilac', 'rich maroon', 'rifle green', 'ripe mango', 'roast coffee', 'robin egg blue', 'rocket metallic', 'roman silver', 'root beer', 'rose', 'rose bonbon', 'rose dust', 'rose ebony', 'rose garnet', 'rose gold', 'rose madder', 'rose pink', 'rose quartz', 'rose quartz pink', 'rose red', 'rose taupe', 'rose vale', 'rosewood', 'rosy brown', 'royal azure', 'royal blue', 'royal blue', 'royal brown', 'royal fuchsia', 'royal green', 'royal orange', 'royal pink', 'royal red', 'royal red', 'royal purple', 'royal yellow', 'ruber', 'rubine red', 'ruby', 'ruby red', 'rufous', 'rum', 'russet', 'russian green', 'russian violet', 'rust', 'rusty red', 'sacramento state green', 'saddle brown', 'safety orange', 'Safety orange ', 'blaze orange', 'safety yellow', 'saffron', 'sage', "st. patrick's blue", 'salem', 'salmon', 'salmon rose', 'salmon pink', 'samsung blue', 'sand', 'sand dune', 'sandstorm', 'sandy brown', 'sandy tan', 'sandy taupe', 'sap green', 'sapphire', 'sapphire blue', 'sasquatch socks', 'satin sheen gold', 'scarlet', 'Scarlet ', 'Crayola', 'schauss pink', 'school bus yellow', "screamin' green", 'sea blue', 'sea foam green', 'sea green', 'Sea green ', 'Crayola', 'sea serpent', 'seal brown', 'seashell', 'selective yellow', 'sepia', 'shadow', 'shadow blue', 'shampoo', 'shamrock green', 'shandy', 'sheen green', 'shimmering blush', 'shiny shamrock', 'shocking pink', 'Shocking pink ', 'Crayola', 'sienna', 'silver', 'Silver ', 'Crayola', 'Silver ', 'Metallic', 'silver chalice', 'silver foil', 'silver lake blue', 'silver pink', 'silver sand', 'sinopia', 'sizzling red', 'sizzling sunrise', 'skobeloff', 'sky blue', 'Sky blue ', 'Crayola', 'sky magenta', 'slate blue', 'slate gray', 'slimy green', 'Smalt ', 'Dark powder blue', 'smashed pumpkin', 'smitten', 'smoke', 'smokey topaz', 'smoky black', 'snow', 'soap', 'solid pink', 'sonic silver', 'spartan crimson', 'space cadet', 'spanish bistre', 'spanish blue', 'spanish carmine', 'spanish crimson', 'spanish gray', 'spanish green', 'spanish orange', 'spanish pink', 'spanish purple', 'spanish red', 'spanish sky blue', 'spanish violet', 'spanish viridian', 'spanish yellow', 'spicy mix', 'spiro disco ball', 'spring bud', 'spring frost', 'spring green', 'Spring green ', 'Crayola', 'star command blue', 'steel blue', 'steel pink', 'steel teal', 'stil de grain yellow', 'straw', 'strawberry', 'stop red', 'strawberry iced tea', 'strawberry red', 'sugar plum', 'sunburnt cyclops', 'sunglow', 'sunny', 'sunray', 'sunset', 'sunset orange', 'super pink', 'sweet brown', 'taffy', 'tan', 'Tan ', 'Crayola', 'tangelo', 'tangerine', 'tangerine yellow', 'tango pink', 'tart orange', 'taupe', 'taupe gray', 'tea green', 'tea rose', 'tea rose', 'teal', 'teal blue', 'teal deer', 'teal green', 'telemagenta', 'temptress', 'Tenné ', 'tawny', 'terra cotta', 'thistle', 'Thistle ', 'Crayola', 'thulian pink', 'tickle me pink', 'tiffany blue', "tiger's eye", 'timberwolf', 'titanium', 'titanium yellow', 'tomato', 'tomato sauce', 'toolbox', 'tooth', 'topaz', 'tractor red', 'trolley grey', 'tropical rain forest', 'tropical violet', 'true blue', 'tufts blue', 'tulip', 'tumbleweed', 'turkish rose', 'turquoise', 'turquoise blue', 'turquoise green', 'turquoise surf', 'turtle green', 'tuscan', 'tuscan brown', 'tuscan red', 'tuscan tan', 'tuscany', 'twilight lavender', 'twitter blue', 'tyrian purple', 'ube', 'ultramarine', 'ultramarine blue', 'Ultramarine blue ', "Caran d'Ache", 'ultra pink', 'ultra red', 'umber', 'unbleached silk', 'united nations blue', 'unmellow yellow', 'up maroon', 'upsdell red', 'urobilin', 'vampire black', 'van dyke brown', 'vanilla', 'vanilla ice', 'vegas gold', 'venetian red', 'verdigris', 'vermilion', 'vermilion', 'veronica', 'verse green', 'very light azure', 'very light blue', 'very light malachite green', 'very light tangelo', 'very pale orange', 'very pale yellow', 'vine green', 'violet', 'Violet ', "Caran d'Ache", 'Violet ', 'color wheel', 'Violet ', 'crayola', 'Violet ', 'RYB', 'Violet ', 'web', 'violet-blue', 'Violet-blue ', 'Crayola', 'violet-red', 'violin brown', 'viridian', 'viridian green', 'vista blue', 'vivid amber', 'vivid auburn', 'vivid burgundy', 'vivid cerise', 'vivid cerulean', 'vivid crimson', 'vivid gamboge', 'vivid lime green', 'vivid malachite', 'vivid mulberry', 'vivid orange', 'vivid orange peel', 'vivid orchid', 'vivid raspberry', 'vivid red', 'vivid red-tangelo', 'vivid sky blue', 'vivid tangelo', 'vivid tangerine', 'vivid vermilion', 'vivid violet', 'vivid yellow', 'water', 'watermelon', 'watermelon red', 'watermelon yellow', 'waterspout', 'weldon blue', 'wenge', 'wheat', 'white', 'white chocolate', 'white coffee', 'white smoke', 'wild orchid', 'wild strawberry', 'wild watermelon', 'willpower orange', 'windsor tan', 'wine', 'wine dregs', 'wine red', 'winter sky', 'winter wizard', 'wintergreen dream', 'wisteria', 'wood brown', 'xanadu', 'yellow', 'Yellow ', 'Crayola', 'Yellow ', 'Munsell', 'Yellow ', 'NCS', 'Yellow ', 'Pantone', 'Yellow ', 'process', 'Yellow ', 'RYB', 'yellow-green', 'Yellow-green ', 'Crayola', 'yellow orange', 'Yellow Orange ', 'Color Wheel', 'yellow rose', 'yellow sunshine', 'yinmn blue', 'zaffre', 'zebra white', 'zinnwaldite', 'zomp']

new_val_info = []
for qa in val_info:
  max_label, max_score = None,None
  for label,score in qa["label"].items():
    if not max_score or score > max_score:
      max_label = label
      max_score = score
  quesType = label2questype(max_label)
  qa["question_type"] = quesType
  new_val_info.append(qa)


new_train_info = []
for qa in train_info:
  max_label, max_score = None,None
  for label,score in qa["label"].items():
    if not max_score or score > max_score:
      max_label = label
      max_score = score
  quesType = label2questype(max_label)
  qa["question_type"] = quesType
  new_train_info.append(qa)

with open('data/vqa/valid.json', 'w') as outfile:
    json.dump(new_val_info, outfile)
with open('data/vqa/train.json', 'w') as outfile:
    json.dump(new_train_info, outfile)




In [ ]:
#1. reformat train.json, val.json 
# !wget https://ivc.ischool.utexas.edu/VizWiz_final/vqa_data/Annotations.zip
# !unzip Annotations.zip
import json
with open('Annotations/train.json') as f1:
  train_json = json.load(f1)
with open('Annotations/val.json') as f2:
  val_json = json.load(f2)

answer_vocabs = set()

new_trains = []
for train in train_json:
  new_train = {}
  new_train["img_id"] = train["image"]
  new_train["question_id"] = train["image"]
  new_train["sent"] = train["question"]
  label_dict = {}
  for ans in train["answers"]:
    answer = ans["answer"]
    answer_vocabs.add(answer)
    label_dict[answer] = min(round(label_dict.get(answer,0) + 0.3, 1), 1.0)
  new_train["label"] = label_dict
  new_trains.append(new_train)
with open('data/vqa/train.json', 'w') as outfile:
    json.dump(new_trains, outfile)


new_vals = []
for val in val_json:
  new_val = {}
  new_val["img_id"] = val["image"]
  new_val["question_id"] = val["image"]
  new_val["sent"] = val["question"]
  label_dict = {}
  for ans in val["answers"]:
    answer = ans["answer"]
    answer_vocabs.add(answer)
    label_dict[answer] = min(round(label_dict.get(answer,0) + 0.3, 1), 1.0)
  new_val["label"] = label_dict
  new_vals.append(new_val)
with open('data/vqa/minival.json', 'w') as outfile:
    json.dump(new_vals, outfile)

print(len(new_trains))
print(len(new_vals))
print(len(answer_vocabs))

20523
4319
48730


In [ ]:
#2. generate ans2label and label2ans file
label2ans = list(answer_vocabs)
with open('data/vqa/trainval_label2ans.json', 'w') as outfile:
    json.dump(label2ans, outfile)
ans2label = {}
for i,ans in enumerate(label2ans):
  ans2label[ans] = i
with open('data/vqa/trainval_ans2label.json', 'w') as outfile:
    json.dump(ans2label, outfile)

!mkdir -p snap/pretrained 
!wget https://nlp.cs.unc.edu/data/model_LXRT.pth -P snap/pretrained

In [ ]:
!mkdir -p data/vizwiz_imgfeat
# !wget https://nlp.cs.unc.edu/data/lxmert_data/vizwiz/vizwiz/valid_obj36.tsv -P data/mscoco_imgfeat
# !wget 'https://s3.us-east-2.amazonaws.com/cmumonkey/36_vizwiz_val.pkl' -P data/vizwiz_imgfeat
!wget 'https://s3.us-east-2.amazonaws.com/cmumonkey/36_vizwiz_train.pkl' -P data/vizwiz_imgfeat

--2021-04-16 02:32:16--  https://s3.us-east-2.amazonaws.com/cmumonkey/36_vizwiz_train.pkl
Resolving s3.us-east-2.amazonaws.com (s3.us-east-2.amazonaws.com)... 52.219.96.106
Connecting to s3.us-east-2.amazonaws.com (s3.us-east-2.amazonaws.com)|52.219.96.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7122021558 (6.6G) [binary/octet-stream]
Saving to: ‘data/vizwiz_imgfeat/36_vizwiz_train.pkl’

36_vizwiz_train.pkl 100%[===================>]   6.63G  75.8MB/s    in 1m 53s  

2021-04-16 02:34:09 (60.2 MB/s) - ‘data/vizwiz_imgfeat/36_vizwiz_train.pkl’ saved [7122021558/7122021558]



In [ ]:
import pickle
with open('data/vizwiz_imgfeat/36_vizwiz_train.pkl', 'rb') as f:
    train_img_feat = pickle.load(f)
with open('data/vizwiz_imgfeat/36_vizwiz_val.pkl', 'rb') as f:
    val_img_feat = pickle.load(f)

In [ ]:
!bash run/vqa_finetune.bash 0 vizwiz_tiny --tiny

In [ ]:
!bash run/vqa_finetune.bash 0 vizwiz_all

In [ ]:
!bash run/vqa_test.bash 0 vizwiz_all_results --test valid --load snap/vqa/vizwiz_all/BEST

train
Load 20523 data from split(s) train.
Start to load Faster-RCNN detected objects from data/vizwiz_imgfeat/36_vizwiz_train.pkl
Loaded 512 images in file data/vizwiz_imgfeat/36_vizwiz_train.pkl in 98 seconds.
Use 444 data in torch dataset

LXRT encoder with 9 l_layers, 5 x_layers, and 5 r_layers.
BertAdam Total Iters: 52
Load model from snap/vqa/vizwiz_all/BEST
valid
Load 4319 data from split(s) valid.
Start to load Faster-RCNN detected objects from data/vizwiz_imgfeat/36_vizwiz_val.pkl
Loaded 7748 images in file data/vizwiz_imgfeat/36_vizwiz_val.pkl in 44 seconds.
Use 4317 data in torch dataset

0.4786889043317073
